In [1]:
# coding: utf-8
"""
作ったライブラリの評価
"""
from filer2.filer2 import Filer
from library.submodular import *
from library.graphsubmodular import *
from library.vector import *
from library.rouge import *
import numpy as np
import collections
from scipy.sparse import csr_matrix, lil_matrix

### 英語版(Opinosis)

In [2]:
# ファイルの読み込み
dict_test = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/test/accuracy_garmin_nuvi_255W_gps.dump')
dict_ans = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/ans/accuracy_garmin_nuvi_255W_gps.dump')

In [3]:
# テスト用ファイルの作成
# １００単語以上ある文は除く
list_sentence = []
list_sep = []
list_sep_all = []
list_edgelist = []
for sen, sep, sepall, edge in zip(dict_test['sentence'],
                                  dict_test['sep_all'],
                                  dict_test['sep_all'],
                                  dict_test['edge_all']):
    if len(sepall) <= 100 and len(sep) != 0:
        list_sentence.append(sen)
        list_sep.append(sep)
        list_sep_all.append(sepall)
        list_edgelist.append(edge)
        
list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]

In [ ]:
# インスタンス化
GM = GraphSubModular(list_sep_all=list_sep_all,
                     list_sep=list_sep,
                     list_edgelist=list_edgelist,
                     directed=True,
                     inverse_flag=True)
V = Vector(list_sep_all=list_sep_all,
           list_sep=list_sep,
           dict_path = './files/OpinosisDataset1.0/w2v/model_lemmas/model_cbow_s300_w5_m0/dict_word_vec_3')

SM = SubModular(list_sep_all=list_sep_all,
                list_sep=list_sep)

MGSM = Modified_GraphSubModular(list_sep_all=list_sep_all,
                                list_sep=list_sep,
                                list_edgelist=list_edgelist,
                                directed=False,
                                simrank_flag=True,
                                weighted=False,
                                log_flag=True)

MV = Modified_Vector(list_sep_all=list_sep_all,
                     list_sep=list_sep,
                     dict_path='./files/OpinosisDataset1.0/w2v/model_tokens/model_cbow_s300_w5_m0/dict_word_vec_3')

In [4]:
MGSM = Multiplex_GraphSubModular(list_sep_all=list_sep_all,
                                 list_sep=list_sep,
                                 weighted=False)

In [ ]:
# 学習
GM.m_greedy(num_w=100, r=1, scale=0)
V.m_greedy(num_w=100, r=1, scale=0)

In [ ]:
SM.m_greedy(num_w=100, alpha=0.1, lamda=6, n_cluster=10)

In [ ]:
MGSM.m_greedy(num_w=100, r=1, scale=0)
MV.m_greedy(num_w=100, r=1, scale=0)

In [7]:
MGSM.m_greedy(num_w=100,
              r=1,
              scale=0,
              a=0.5,
              b=0.5)

計算が終了しました


In [ ]:
# 文書数のチェック
print 'GM:'
print 'sentence: ', len(GM.list_C)
for row in GM.list_C:
    print len(row[2])
print 'V:'
print 'sentence: ', len(V.list_C)
for row in V.list_C:
    print len(row[2])
print 'SM:'
print 'sentence: ', len(SM.list_C)
for row in SM.list_C:
    print len(row[2])
print 'MGSM:'
print 'sentence: ', len(MGSM.list_C)
for row in MGSM.list_C:
    print len(row[2])
print 'MV:'
print 'sentence: ', len(MV.list_C)
for row in MV.list_C:
    print len(row[2])

In [8]:
print 'MGSM:'
print 'sentence: ', len(MGSM.list_C)
for row in MGSM.list_C:
    print len(row[2])

MGSM:
sentence:  property: list_C
9
property: list_C
5
15
9
15
8
14
15
11
6


In [ ]:
for row in GM.list_C:
    print row[0]
for row in V.list_C:
    print row[0]
for row in SM.list_C:
    print row[0]
for row in MGSM.list_C:
    print row[0]
for row in MV.list_C:
    print row[0]

In [ ]:
# 単語数のチェック
counter = 0
for row in SM.list_C:
    counter += len(row[2])
print counter

In [ ]:
# Rougeを計算するようにbag_of_wordsを作成する
list_gm_test = [row[2] for row in GM.list_C]
list_v_test = [row[2] for row in V.list_C]
list_sm_test = [row[2] for row in SM.list_C]
list_mgm_test = [row[2] for row in MGM.list_C]
list_mv_test = [row[2] for row in GM.list_C]

In [ ]:
num=4
print Rouge_N.rouge(list_gm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_v_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_sm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_mgm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_mv_test, dict_ans['sep_all'][num], N=1)

### 品詞ごとに最適化

In [1]:
from filer2.filer2 import Filer
from library.submodular import *
from library.graphsubmodular import *
from library.vector import *
from library.rouge import *
import numpy as np
import collections
from scipy.sparse import csr_matrix, lil_matrix

In [13]:
# ファイルの読み込み
dict_test = Filer.readdump('./files/OpinosisDataset1.0/preprocessed3/bathroom_bestwestern_hotel_sfo.dump')
dict_ans = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/ans/bathroom_bestwestern_hotel_sfo.dump')

In [14]:
# テスト用ファイルの作成
# １００単語以上ある文は除く
list_sentence = []
list_sep = []
list_sep_all = []
list_pos = []
list_edgelist = []
for sen, sep, sepall, pos, edge in zip(dict_test['sentence'],
                                       dict_test['sep_all'],
                                       dict_test['sep_all'],
                                       dict_test['pos_all'],
                                       dict_test['edge_all']):
    if len(sepall) <= 100 and len(sep) != 0:
        list_sentence.append(sen)
        list_sep.append(sep)
        list_sep_all.append(sepall)
        list_pos.append(pos)
        list_edgelist.append(edge)
        
list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]

In [15]:
PGSM = Partition_GraphSubModular(list_sep_all=list_sep_all,
                                 list_sep=list_sep,
                                 list_pos=list_pos,
                                 list_edgelist=list_edgelist,
                                 directed=False,
                                 simrank_flag=True,
                                 weighted=False,
                                 log_flag=True)

In [16]:
PGSM.m_greedy(num_w = 100,
              r=1,
              scale=0,
              n=0.2,
              j=0.2,
              v=0.2,
              o=0.2)

計算が終了しました


In [17]:
list_C = PGSM.list_C
list_pgsm_test = [list_sentence[row[0]].replace(',', '').replace('.', '').lower().split(' ') for row in list_C]
list_pgsm_ans = [[sen.replace(',', '').replace('.', '').lower().split(' ') for sen in row] for row in dict_ans['sentence']]
R1 = Rouge_N.rouge(list_pgsm_test, list_pgsm_ans, N=1)
print list_pgsm_test
print list_pgsm_ans
print R1

property: list_C
[[u'bathroom', u'spotless', u'and', u'nicely', u'appointed'], [u'large', u'comfortable', u'room', u'wonderful', u'bathroom'], [u'the', u'rooms', u'are', u'exceptionally', u'clean', u'and', u'also', u'the', u'bathrooms'], [u'shower', u'bathroom', u'was', u'good', u'with', u'good', u'shower', u'and', u'plenty', u'of', u'space'], [u'hair', u'dryer', u'in', u'bathroom', u'constantly', u'fell', u'in', u'sink', u'from', u'it', u"'s", u'holder', u'when', u'anyone', u'used', u'the', u'door', u'to', u'our', u'room'], [u'bathrooms', u'though', u'are', u'plenty', u'big'], [u'amenities', u'while', u'this', u'hotel', u'is', u"n't", u'a', u'luxury', u'property', u'they', u'have', u'some', u'nice', u'touches', u'like', u'marble', u'bathrooms', u'mini', u'bar', u'room', u'service', u'and', u'an', u'evening', u'wine', u'reception', u'every', u'day'], [u'the', u'room', u'was', u'small', u'with', u'a', u'king', u'bed', u'and', u'a', u'truly', u'claustrophobic', u'bathroom']]
[[[u'bathroo

In [6]:
num = 100

print PGSM._d_matrix.shape
print PGSM._d_matrix_n.shape
print PGSM._d_matrix_j.shape
print PGSM._d_matrix_v.shape
print PGSM._d_matrix_o.shape

print PGSM._d_matrix[:,num].sum()
print PGSM._d_matrix_n[:,num].sum()
print PGSM._d_matrix_j[:,num].sum()
print PGSM._d_matrix_v[:,num].sum()
print PGSM._d_matrix_o[:,num].sum()

(1191, 436)
(248, 436)
(129, 436)
(224, 436)
(590, 436)
45.1255534475
7.52334787
4.46805478063
7.97308542162
25.1610653753
